In [1]:
# Run these lines only the first time you run this notebook

# !pip install tensorflow_addons gdown shap -q
# !pip install git+https://github.com/katarinagresova/DeepExperiment

# !wget https://github.com/ML-Bioinfo-CEITEC/miRBind/raw/main/Models/miRBind.h5
# !wget https://raw.githubusercontent.com/ML-Bioinfo-CEITEC/miRBind/graphs/Datasets/evaluation_set_1_1_CLASH2013_paper.tsv

In [2]:
import tensorflow

In [3]:
tensorflow.__version__

'2.9.0'

### Load and preprocess the transcript data

In [4]:
from Bio import SeqIO
import re

def extract_gene_symbol(record):
    tmp = [re.findall("\((.*?)\)", x) for x in record.description.split()]
    while([] in tmp):
        tmp.remove([])
    return tmp[0][0]

gene_symbol_to_seq = {}
file_path = 'data/GCF_000001405.40/rna.fna'

for record in SeqIO.parse(file_path, "fasta"):
    description = record.description.split()
    gene_id = next((item.split('=')[1] for item in description if item.startswith('gene=')), None)
    
    symbol = extract_gene_symbol(record)
    # Do not need the locus tag (== record.id) for now
    # gene_symbol_to_seq[symbol] = [record.seq, record.id]
    gene_symbol_to_seq[symbol] = record.seq    

In [5]:
len(gene_symbol_to_seq.items())

42423

### Get our deep learning model and miRNA data

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import register_keras_serializable
import tensorflow_addons as tfa
import cv2 
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from textwrap import wrap
import random
import shap
from shap.plots.colors import red_transparent_blue

from deepexperiment.alignment import Attrament
from deepexperiment.visualization import plot_alignment, plot_miRNA_importance, plotbar_miRNA_importance
from deepexperiment.utils import one_hot_encoding, one_hot_encoding_batch
from deepexperiment.interpret import DeepShap

/home/jovyan/my-conda-envs/deepExperimentTF2.7/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/jovyan/my-conda-envs/deepExperimentTF2.7/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonote

In [7]:
random.seed(42)

In [8]:
"""# Loading model and the data"""

model = keras.models.load_model("miRBind.h5")
# model.summary()

samples = pd.read_csv('evaluation_set_1_1_CLASH2013_paper.tsv', sep='\t')
samples.head()

# get the miRNAs
#     CHANGE choose the miRNA
# miRNA = samples['miRNA'].value_counts().index[6]
# miRNA

2023-08-15 11:50:35.892940: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 11:50:37.093918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79109 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:61:00.0, compute capability: 8.0


,miRNA,gene,label
0,TAGGTAGTTTCATGTTGTTG,AGACCAGCAACAAACTACCAACACTGTAGAGGAGCCCCTGGATCTT...,1
1,GCAGTCCATGGGCATATACA,CAGCCAAGCCCAGGCTCGCAGTATCAATAAAGCAGTTCTCTGAGGA...,1
2,CAAAGTGCTTACAGTGCAGG,TGAGAACACTTAAAGTTCAAATAGAAATCATTTCTGAAGACAAAAG...,1
3,GAGGGTTGGGTGGAGGCTCT,AATGCCCCAAAATCCAAACCTTCCTGAACGCTATGACACCATGAGT...,1
4,ACTGCCCTAAGTGCTCCTTC,AAGAAACTAGGAGAATGGGTAGGCCTTTGTAAAATTGACAGAGAGG...,1



### Use and evaluate the model

In [9]:
# n = 50 is number of samples used as a background image to compare the input with during the shap method
rand_samples = samples.sample(n=50, replace=False, random_state=42).reset_index(drop=True)
background, _ = one_hot_encoding_batch(rand_samples)
deepShap = DeepShap(model, background)

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
2023-08-15 11:50:39.416965: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204
2023-08-15 11:50:41.077545: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [10]:
# from numpy import random

def score_sequence_attribution_minimal(gene, input_miRNA, model, draw_plot=False, step=10, length=50):
    # TODO quick fix: tensor_dim argument in one_hot_encoding_batch() and the whole model expects RNA of len 20
    miRNA = input_miRNA[0:20]
    
    miRNAs = []
    genes = []

    for i in range(0, len(gene) - length, step):
        start = max(i, 0)
        end = min(i+50, len(gene))
        miRNAs.append(miRNA)
        genes.append(gene[start:end])

    labels = np.zeros(len(genes))

    df = pd.DataFrame(
        {'miRNA': miRNAs,
         'gene': genes,
         'label': labels
        })
    data, _ = one_hot_encoding_batch(df, tensor_dim=(50, 20, 1))
    preds = model(data)

    attribution = np.zeros((len(gene), len(miRNA)))
    shap_indices = []
    pred_indices = []
    preds_indices = []
    shap_data = []

  # counter == index to predictions for windows
    counter = 0
    for i in range(0, len(gene) - length, step):
        if preds[counter][1] > 0.5:
            # shap_indices == indices of positively predicted sliding windows (indexes preds array)
            shap_indices.append(counter)
            pred_indices.append(i)
            shap_data.append(data[counter])
        counter+=1
    
    # TODO quick fix: if at this point the 'shap_data' is empty array [], it means no prediction was > 0.5, therefore (for now) let's predict 0 binding affinity  
    if len(shap_data) == 0:
        return []
    
    shap_data = np.stack(shap_data)
    # TODO: batching?
    # using only pos_shap cos we only care about positives, pos_shap and neg_shap are opposites for binary classification
    neg_shap, pos_shap = deepShap(shap_data)

    for i in range(0, len(shap_indices)):

        normalized_shap = pos_shap[i,:,:,0] * preds[shap_indices[i]][1]

        # padding current shap-value-matrix to sum it up to the cumulative attribution matrix
        newrows = np.zeros((pred_indices[i], normalized_shap.shape[1]))
        normalized_shap = np.vstack([newrows, normalized_shap])
        newrows = np.zeros((len(gene) - pred_indices[i] - length, normalized_shap.shape[1]))
        normalized_shap = np.vstack([normalized_shap, newrows])

        attribution += normalized_shap

    attribution = attribution.T.max(axis=0)
    # Fix `counts = ...` for genes shorter than 98 nucleotides
    # pad the `attribution` or truncate `counts`
    counts = list(range(1, 50)) + (len(attribution) - 98) * [50] + list(range(49, 0, -1))
    if len(attribution) < 98:
        counts = counts[:len(attribution)]
        # plt.figure(num = random.randint(0, 1000)) # num is a unique identifier for the figure
        # plt.plot(counts)
        # plt.plot(attribution * 1000)
        # plt.show()
    
    try:            
        normalized_scores = attribution / np.array(counts)
    except Exception as e:
        print(e)
        print(miRNA)
        print(gene)
        print(len(attribution), len(np.array(counts)))
        normalized_scores = 0
    

    if draw_plot:
        #     TODO change printing into a single plot?
        plt.figure(num = random.randint(0, 1000)) # num is a unique identifier for the figure
        plt.plot(normalized_scores);
    
    return normalized_scores

### Get the scores for FC comparison using files from Bartel  

In [11]:
mirna_FCs = pd.read_csv('modules/evaluation/mirna_fcs.csv',index_col=0, header=0, sep=',')

In [12]:
# mirna_FCs
mirna_FCs['Gene symbol']

RefSeq ID
NM_000017       ACADS
NM_000019       ACAT1
NM_000021       PSEN1
NM_000023        SGCA
NM_000027         AGA
               ...   
NM_213646        WARS
NM_213647       FGFR4
NM_213649       SFXN4
NM_213651    SLC25A24
NM_213662       STAT3
Name: Gene symbol, Length: 8372, dtype: object

In [13]:
def rna_to_dna(rna_sample):
    rna_dic = {
        'A':'A',
        'C':'C',
        'U':'T',
        'G':'G',
    }
    converted = [rna_dic[x.upper()] for x in rna_sample]
    new = ""
    # traverse in the string
    for x in converted:
        new += x
    # return string
    return new

In [14]:
mirna_sequences = ['UAGCAGCACGUAAAUAUUGGCG', 'UAAAGUGCUGACAGUGCAGAU', 'UAACACUGUCUGGUAACGAUGU', 'UAAUACUGCCUGGUAAUGAUGA', 'AUGACCUAUGAAUUGACAGAC', 'UGAGGUAGUAGGUUGUAUGGUU', 'AGCAGCAUUGUACAGGGCUAUGA']
mirna_sequences = [rna_to_dna(x) for x in mirna_sequences]
print(mirna_sequences)

['TAGCAGCACGTAAATATTGGCG', 'TAAAGTGCTGACAGTGCAGAT', 'TAACACTGTCTGGTAACGATGT', 'TAATACTGCCTGGTAATGATGA', 'ATGACCTATGAATTGACAGAC', 'TGAGGTAGTAGGTTGTATGGTT', 'AGCAGCATTGTACAGGGCTATGA']


In [15]:
miRNA_names = ['hsa-miR-16-5p', 'hsa-miR-106b-5p', 'hsa-miR-200a-3p', 'hsa-miR-200b-3p', 'hsa-miR-215-5p', 'hsa-let-7c-5p', 'hsa-miR-103a-3p']
miRNA_name_to_seq = {}
for i in range(len(miRNA_names)):
    miRNA_name_to_seq[miRNA_names[i]] = mirna_sequences[i]
miRNA_name_to_seq

{'hsa-miR-16-5p': 'TAGCAGCACGTAAATATTGGCG',
 'hsa-miR-106b-5p': 'TAAAGTGCTGACAGTGCAGAT',
 'hsa-miR-200a-3p': 'TAACACTGTCTGGTAACGATGT',
 'hsa-miR-200b-3p': 'TAATACTGCCTGGTAATGATGA',
 'hsa-miR-215-5p': 'ATGACCTATGAATTGACAGAC',
 'hsa-let-7c-5p': 'TGAGGTAGTAGGTTGTATGGTT',
 'hsa-miR-103a-3p': 'AGCAGCATTGTACAGGGCTATGA'}

#### Only for 1 miRNA now 

In [16]:
mirna_sequences = ['TAAAGTGCTGACAGTGCAGAT']
mirna_sequences
mirna_name = 'hsa-miR-106b-5p'
# mirna_name = 'hsa-miR-16-5p'
my_miRNA = 'TAAAGTGCTGACAGTGCAGAT'
# my_miRNA = 'TAGCAGCACGTAAATATTGGCG'

In [17]:
gene_symbols = mirna_FCs[mirna_FCs[mirna_name].isnull() == False]['Gene symbol'].values
print(len(gene_symbols))

8092


#### Set `threshold_len` to filter out longer genes

In [18]:
threshold_len = -1

if threshold_len > 0:
    count = 0
    for key,value in gene_symbol_to_seq.items():
        if(len(value) < threshold_len):
            count += 1
    count

In [19]:
if threshold_len > 0:
    count_too_long = 0
    count_kept = 0
    for key,value in gene_symbol_to_seq.items():
        if(len(value) > threshold_len):
            gene_symbol_to_seq[key] = []
            count_too_long += 1
        else:
            count_kept += 1
    print('count_too_long, len > ', threshold_len, " : ", count_too_long)
    print('count_kept ', count_kept)

### Scan the transcript

In [20]:
import time

# two objects to collect results for now, will see which one is more handy
score_table = []
miRNA_to_gene_score = {}
explain_errors = []

start = time.time()

i = 0
for miRNA in mirna_sequences[:1]:
    print(miRNA)
    if miRNA not in miRNA_to_gene_score:
        miRNA_to_gene_score[miRNA] = []
    for gene_symbol, gene_sequence in gene_symbol_to_seq.items():
        if len(gene_sequence) > 0:
            # print(len(value), len(miRNA))
            if len(miRNA) > 22:
                print(miRNA)
            try:
                score = score_sequence_attribution_minimal(gene_sequence, miRNA, model, draw_plot=False)
                score_table.append([miRNA, gene_symbol, score])
                miRNA_to_gene_score[miRNA].append([gene_symbol, score])
            except (AssertionError, ValueError) as e:
                print(miRNA, gene_symbol, e)
                explain_errors.append([miRNA, gene_symbol, e])
                
        else:
            explain_errors.append(gene_symbol)
        
        i+=1
        if i % 500 == 0:
            print(gene_symbol, " |",i , "| " , end =" ")
        # if i > 1000:
        #     break
    break
        
            
end = time.time()
print(end - start, ' seconds')

TAAAGTGCTGACAGTGCAGAT


`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


TAAAGTGCTGACAGTGCAGAT LAMA2 The SHAP explanations do not sum up to the model's output! This is either because of a rounding error or because an operator in your computation graph was not fully supported. If the sum difference of 0.041885 is significant compared the scale of your model outputs please post as a github issue, with a reproducable example if possible so we can debug it.
PMS2  | 500 |  

KeyboardInterrupt: 

In [21]:
# score_table[:1]

In [22]:
empties = 0
for gene_n_score in miRNA_to_gene_score[my_miRNA]:
    if len(gene_n_score[1]) == 0:
        empties += 1
print(empties, ' / ', len(miRNA_to_gene_score[my_miRNA]))

189  /  750


#### Save the explainability scoring to a file

In [23]:
for key, value in miRNA_name_to_seq.items():
    if value == my_miRNA:
        print(key)
        my_miRNA_name = key

hsa-miR-106b-5p


In [24]:
print(type(miRNA_to_gene_score), type(miRNA_to_gene_score[my_miRNA]))

<class 'dict'> <class 'list'>


In [25]:
import json

save_scores_path = "explainability_scores_{}.json".format(my_miRNA_name)

SAVE_EXPL_SCORES = False
if SAVE_EXPL_SCORES:
    with open(save_scores_path, 'w') as file:
        data_to_save = {}
        for key in miRNA_to_gene_score.keys():
            data_to_save[key] = [[sub_key, list(sub_val)] for sub_key, sub_val in miRNA_to_gene_score[key]]

        json.dump(data_to_save, file, indent=4)

In [26]:
len(explain_errors)

1

#### Load the explainability scoring

In [27]:
load_scores_path = save_scores_path

with open(load_scores_path, 'r') as file:
    miRNA_to_gene_score_loaded = json.load(file)

In [28]:
print(type(miRNA_to_gene_score_loaded), type(miRNA_to_gene_score_loaded[my_miRNA]))

<class 'dict'> <class 'list'>


In [29]:
loaded_genes = [x[0] for x in miRNA_to_gene_score_loaded[my_miRNA]]

### Plot

In [ ]:
# gene_symbol_to_score = {}

# for i in miRNA_to_gene_score_loaded[my_miRNA]:
#     gene_symbol = i[0]
#     score = i[1]
#     gene_symbol_to_score[gene_symbol] = score

In [ ]:
from modules.collect_scores.bymean import bymeanfunc
from modules.collect_scores.bysum import bysumfunc
from modules.collect_scores.bymaxvalue import bymaxvaluefunc
import math

# from modules.collect_scores.simple_methods import bymaxvaluefunc, bysumfunc, bymeanfunc

gene_score=[]
# for gene,score in gene_to_score.items():
for gene_symbol, score_signal in miRNA_to_gene_score_loaded[my_miRNA]:
    if len(score_signal) > 0:
        prediction = bymaxvaluefunc(score_signal)
        gene_score.append([gene_symbol, prediction])
    else:
        gene_score.append([gene_symbol, 0])
        
        # "-1 * x" because we compare with fold change where lower == more impact
        # prediction = -1 * bymaxvaluefunc(gene_to_score[gene])
        # prediction = math.log(bymaxvaluefunc(gene_to_score[gene]))
        
        # gene : [real, prediction] 
        # gene_to_comparison[gene] = [gene_to_fc[gene], prediction]
        

In [ ]:
results_df = pd.DataFrame(gene_score, columns=["Gene Symbol","score"])
results_df[results_df['score'] > 0]

In [ ]:
all_genes = []
for gene_symbol, gene_sequence in gene_symbol_to_seq.items():
    all_genes.append(gene_symbol)

#### (prediction) for genes
X axis is different genes, Y axis is fold change

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt

###Importing dummy data
#Predictions
targetscan = pd.read_csv('Predicted_Targets_Context_Scores.default_predictions.txt',index_col=0, header=0, sep='\t')
targetscan = targetscan[["context++ score","weighted context++ score","miRNA","Gene Symbol"]]
targetscan = targetscan[targetscan['miRNA'] == mirna_name]
#Expression
mirna_FCs = pd.read_csv('mirna_fcs.csv',index_col=0, header=0, sep=',')


predictions = {}

predictions["context++ score"] = targetscan[["context++ score","miRNA","Gene Symbol"]]
predictions["context++ score"] = predictions["context++ score"].rename(columns={'context++ score': 'score'})
predictions["weighted context++ score"] = targetscan[["weighted context++ score","miRNA","Gene Symbol"]]
predictions["weighted context++ score"] = predictions["weighted context++ score"].rename(columns={'weighted context++ score': 'score'})


In [ ]:
predictions["context++ score"]

In [ ]:
merged_df = pd.merge(predictions["context++ score"],
                             mirna_FCs[["Gene symbol",mirna_name]],
                             left_on='Gene Symbol',
                             right_on='Gene symbol',
                             how='left').sort_values(mirna_name,
                                                     ascending=True).dropna()
merged_df
bartel_df = merged_df

In [ ]:
results_df_merged = pd.merge(results_df,
                             mirna_FCs[["Gene symbol",mirna_name]],
                             left_on='Gene Symbol',
                             right_on='Gene symbol',
                             how='left').sort_values(mirna_name,
                                                     ascending=True)
                                                     # ascending=True).dropna()

results_df_merged

### Comparison 

In [ ]:
results_df_merged_notnull = results_df_merged[results_df_merged['hsa-miR-106b-5p'].notnull()]
results_df_merged_notnull 

In [ ]:
results_df_merged['score'].corr(results_df_merged[mirna_name])

In [ ]:
bartel_df['score'].corr(bartel_df[mirna_name])

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# assuming y_true is your ground truth values and y_pred are your predicted values
# both y_true and y_pred are lists or numpy arrays
y_true = bartel_df[mirna_name].to_numpy()
y_pred = bartel_df['score'].to_numpy()

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("MSE: ", mse)
print("R^2: ", r2)


In [ ]:
y_true = results_df_merged_notnull['score'].notnull().to_numpy()
y_pred = results_df_merged_notnull[mirna_name].notnull().to_numpy()

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("MSE: ", mse)
print("R^2: ", r2)

In [ ]:
mirna_FCs[["Gene symbol"]].to_numpy()

In [ ]:
len(set(all_genes).intersection(set(mirna_FCs[["Gene symbol"]].to_numpy().flatten())))

In [ ]:
results_df['Gene Symbol'].to_numpy()

In [ ]:
mirna_FCs

In [ ]:
results = {}
for algorithm in predictions:
        results[algorithm] = pd.DataFrame(columns=["Targets","Mean_mRNA_FC"])
        merged_df = pd.merge(predictions[algorithm],
                             mirna_FCs[["Gene symbol",mirna_name]],
                             left_on='Gene Symbol',
                             right_on='Gene symbol',
                             how='left').sort_values("score",
                                                     ascending=True).dropna()
        
        
        for i,targets in enumerate([2**n for n in range(2, 11)]):
            tmp = merged_df.head(targets)
            meanFC = tmp[mirna_name].mean()
            results[algorithm].loc[i,"Targets"] = targets
            results[algorithm].loc[i,"Mean_mRNA_FC"] = meanFC

In [ ]:
fig = go.Figure()

for algorithm in results:
    fig.add_trace(go.Scatter(x=results[algorithm]['Targets'].index.tolist(),
                                 y=results[algorithm]['Mean_mRNA_FC'],
                                 mode='lines+markers',
                                 name=algorithm)
                                 )


    # create the layout for the plot
fig.update_layout(xaxis=dict(tickvals=results[algorithm]['Targets'].index.tolist(),
                             ticktext=results[algorithm]['Targets'],
                             title = 'Top predicted targets'),
                  yaxis=dict(title='Mean_mRNA_FC'),
                  title=f'Benchmark on {mirna_name} dataset')
    
    # show the plot
fig.show()
# iplot(fig)               # use plotly.offline.iplot for offline plot

### Compare with FC

In [ ]:
def get_scores_by_miRNA(miRNA, score_table):
    result = {}
    for row in score_table:
        if row[0] == miRNA:
            result[row[1]] = row[2] 
    return result

In [ ]:
miRNA_chosen = score_table[0][0]
gene_to_score = get_scores_by_miRNA(
    miRNA_chosen, 
    score_table
)

miRNA_chosen_name = [i for i in miRNA_name_to_seq if miRNA_name_to_seq[i]==miRNA_chosen]
miRNA_chosen_name = miRNA_chosen_name[0]
print('scores for miRNA: ', miRNA_chosen_name, " : ", miRNA_chosen)

In [ ]:
# genes_for_miRNA = mirna_FCs[mirna_FCs[miRNA_chosen_name].isnull() == False]['Gene symbol'].values

In [ ]:
fc_for_miRNA = mirna_FCs[mirna_FCs[miRNA_chosen_name].isnull() == False][['Gene symbol', miRNA_chosen_name]].values

In [ ]:
fc_for_miRNA

In [ ]:
gene_to_fc = {fc_for_miRNA[i][0] : fc_for_miRNA[i][1] for i in range(len(fc_for_miRNA))}

In [ ]:
gene_to_comparison

In [ ]:
import numpy as np

predictions = [x[1] for x in gene_to_comparison.values()]

#normalize all values to be between 0 and 1
predictions_norm = (predictions-np.min(predictions))/(np.max(predictions)-np.min(predictions))
predictions_norm

In [ ]:
pred_min = np.min(predictions)
pred_max = np.max(predictions)

gene_to_comparison_norm = {}
for key,value in gene_to_comparison.items():
    norm_value = (value[1]-pred_min)/(pred_max-pred_min)
    # if norm_value == 0:
    #     norm_value = math.log(0.001)
    # else:
    #     norm_value = math.log(norm_value)
    gene_to_comparison_norm[key] = [value[0], norm_value]

In [ ]:
gene_to_comparison_norm

#### (FC and prediction) for genes
X axis is different genes, Y axis is fold change

In [ ]:
fc_vis = [x[0] for x in list(gene_to_comparison.values())]
preds_vis = [x[1] for x in list(gene_to_comparison.values())]
print(fc_vis, preds_vis)

In [ ]:
plt.figure(num = random.randint(0, 1000)) # num is a unique identifier for the figure
plt.scatter(fc_vis, preds_vis)

In [ ]:
predictions = [x[1] for x in gene_to_comparison.values()]


plt.figure(num = random.randint(0, 1000)) # num is a unique identifier for the figure
plt.plot(predictions);